# Models
> Ad hoc collection of models.

In [ ]:
#| default_exp models

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from wafer.basics import *

## Input convex

In [ ]:
#| export
from wafer.init import default_init, lambda_init

In [ ]:
#| export
class FICNN(nn.Module):
    "Fully input-convex NN. Refer to [ICNN](https://arxiv.org/abs/1609.07152)."
    def __init__(self, ni: int,                        # input size
                 nh: Union[int, list[int]],            # hidden size
                 no: int,                              # output size
                 num_layer: int=2,                     # number of layers (include the output layer), if `nh` is a list, then the `num_layer = len(nh) + 1`
                 actn: Union[str, nn.Module]='relu',   # hidden activation
                 out_actn: Union[str, nn.Module]=None, # output activation
                 init_gain: float=1.                   # weight initialization gain
                ):
        super().__init__()
        nhs = [nh] * (num_layer - 1) if isinstance(nh, int) else nh
        self.w_y = nn.ModuleList([nn.Linear(ni, nh) for nh in nhs + [no]])
        self.w_z = nn.ModuleList([nn.Linear(i, j, bias=False) for i,j in zip(nhs, nhs[1:] + [no])])
        self.actn = get_actn(actn, nn.ReLU())
        self.out_actn = get_actn(out_actn)

        # Initialize
        lambda_init(self, lambda w,b: (nn.init.xavier_normal_(w, init_gain), nn.init.zeros_(b)))
        self.weight_constraint()
    
    def weight_constraint(self):
        "Apply nonnegative weight constriant."
        with torch.no_grad():
            for n,p in self.w_z.named_parameters():
                if 'weight' in n:
                    p.clamp_min_(0)
    
    def forward(self, x):
        z = self.w_y[0](x)
        if len(self.w_z) == 0: return self.out_actn(z)
        for wz,wy in zip(self.w_z[:-1], self.w_y[1:-1]):
            z = self.actn(wz(z) + wy(x))
        z = self.out_actn(self.w_z[-1](z) + self.w_y[-1](x))
        return z

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()